In [2]:
import math
import numpy as np
import torch
from torch import nn
from torch.utils.data import Dataset,DataLoader
import re
from torch.nn import functional as F



In [ ]:
split_ratio = 0.9

# model params
batch_size = 128 # b, to be changed
sequence_l = 128 # n
d_model = 768 # d_model， embedding dim
num_layer = 12 # number of blocks stacked
number_head = 8 # multihead attention
d_ff = 2048 # feedforward dimension
dropout = 0.2
learning_rate = 1e-4
max_epoch = 3

device = 'cuda' if torch.cuda.is_available() else 'cpu'


In [4]:
def build_look_up(text): 
    chars = sorted(list(set(text)))
    stoi = { ch:i for i,ch in enumerate(chars) }
    itos = { i:ch for i,ch in enumerate(stoi) }
    return stoi,itos

def encode(text, stoi):
    return [stoi[c] for c in text]

def decode(idx, itos):
    return [itos[i] for i in idx]

In [5]:
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train if split == 'train' else val
    ix = torch.randint(len(data) - sequence_l, (batch_size,1))
    x = torch.stack([data[i:i+sequence_l] for i in ix])
    y = torch.stack([data[i+1:i+sequence_l+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

# get data and preprocess

In [6]:
with open('data.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# look up and inerse lookup
stoi,itos = build_look_up(text)

# encode data 
data = torch.tensor(encode(text,stoi),dtype=torch.long)

# split train and test
train = data[:int(split_ratio*len(data))]
val = data[int(split_ratio*len(data)):]

# get batch of data
x,y = get_batch('train')

# model 

In [7]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        # pe: [seq_lens * 1 * d_model] for each sample

        self.register_buffer('pe', pe)

    def forward(self, x):
        """
        Arguments:
            x: Tensor, shape ``[seq_len, batch_size, embedding_dim]
        """
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

In [8]:
class Head(nn.Module):
    def __init__(self,head_size):
        super().__init__()
        self.head_size = head_size
        self.key = nn.Linear(d_model,head_size,bias=False)
        self.query = nn.Linear(d_model,head_size,bias=False)
        self.value = nn.Linear(d_model,head_size,bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(sequence_l, sequence_l)))


    def forward(self,x): # x:[batch, l_seq, d_model]
        k = self.key(x) # k:[batch, l_seq, head_size]
        q = self.query(x) # q:[batch, l_seq, head_size]
        v = self.value(x) # v:[batch, l_seq, head_size]
        qkt = q@k.transpose(2,1)/self.head_size**0.5 #[batch*l_seq*l_seq]  
        qkt = qkt.masked_fill(self.tril == 0, float('-inf'))
        qkt = F.softmax(qkt, dim = -1)
        z = qkt@v # z:[batch * l_seq*l_seq]@[batch, l_seq, head_size] = [batch, l_seq, head_size]
        return z




In [9]:
class MultiHeadAttention(nn.Module):
    def __init__(self,number_head,head_size):
        super().__init__()
        self.self_attention = nn.ModuleList([Head(head_size) for _ in range(number_head)])
        self.w0 = nn.Linear(head_size*number_head,d_model)

    def forward(self,x):
        head_outputs = [head(x) for head in self.self_attention]
        output = torch.cat(head_outputs, dim=-1) # [batch, l_seq, head_size*number_head]
        output = self.w0(output) # output:[batch, l_seq, d_model], so that it can be added with residual 
        return output
        

In [10]:
# The Multi-Heads Self-Attention mechanism is followed by two fully connected layers of 
# the Feed Forward block. The first (hidden) layer contains 4 times as many neurons as the input 
# sequence with the ReLU activation function. The dimension of the second layer is 
# equal to the dimension of the input sequence, and neurons do not use the activation function.
class FeedForward(nn.Module):
    def __init__(self,d_model):
        super().__init__()
        self.ff=nn.Sequential( nn.Linear(d_model,4*d_model),
                              nn.ReLU(),
                              nn.Linear(4*d_model,d_model))
    def forward(self,x):
        x = self.ff(x)
        return x

In [11]:
class Block(nn.Module):
    def __init__(self):
        super().__init__()
        head_size = d_model // number_head
        
        self.self_attention = MultiHeadAttention(number_head,head_size)
        
        self.norm1  = nn.LayerNorm(d_model)
        
        self.ffn = FeedForward(d_model)
        
        self.norm2  = nn.LayerNorm(d_model)###
        
    def forward(self,x):
        x = x + self.self_attention(self.norm1(x))
        out = x + self.ffn(self.norm2(x))
         
        return out    





In [12]:
# Model
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.tok_emb = nn.Embedding(len(stoi),d_model)
        self.pos_emb = PositionalEncoding(d_model)
        self.dropout1 = nn.Dropout(dropout)
        
        self.blocks = nn.Sequential(*[Block() for _ in range(num_layer)])
        self.norm_final = nn.LayerNorm(d_model)
        self.predict = nn.Linear(d_model,len(stoi))
        self.loss_compute = nn.CrossEntropyLoss()

    def forward(self, x, use='train',y = None ):
        emb_x = self.tok_emb(x)
        emb_x = self.pos_emb(emb_x) # x,y = emb = [batch size * sequence_l * d_model]
        emb_x = self.dropout1(emb_x)

        emb_x = self.blocks(emb_x)
        x = self.norm_final(emb_x)
        logit = self.predict(x) #[batch size * sequence_l * number_of_char]
        # y:[batch size * l_sequence * 1]

        if use == 'train':
            logit = logit.view(batch_size*sequence_l,len(stoi))
            y = y.view(batch_size*sequence_l)
            loss = self.loss_compute(logit,y)
        elif use == 'generate':
            loss = None

        return logit, loss # loss for training, logit for generate       
    
    def generate(self, output_length, seed_idx):
        for i in range(output_length):
            logit,_ = self(seed_idx, use = 'generate')
            prob = F.softmax(logit)
            next_idx = prob.argmax()
            seed_idx = torch.cat(seed_idx,next_idx)
        return seed_idx


            
            

In [13]:
model = Model()
m = model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for epoch in range(max_epoch):
    x,y = get_batch('train')
    logit,loss = m(x,'train', y)
    loss.backward()
    optimizer.step()
    print(loss)

tensor(4.3660, grad_fn=<NllLossBackward0>)
tensor(4.6809, grad_fn=<NllLossBackward0>)
tensor(3.5571, grad_fn=<NllLossBackward0>)
tensor(3.9997, grad_fn=<NllLossBackward0>)


KeyboardInterrupt: 

# Result generation

In [47]:
seed = text[:10]
stoi[' ']
empty = " "*sequence_l
seed_idx = encode(seed,stoi)
if len(seed)<len(input):
    input_idx = encode(input,stoi)
    input_idx[-len(seed):] = seed_idx
else: 
    input_idx = seed_idx[:sequence_l]

print(len(input_idx))  
print(input_idx)


[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 18, 47, 56, 57, 58, 1, 15, 47, 58, 47]
128
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 18, 47, 56, 57, 58, 1, 15, 47, 58, 47]


In [44]:
text[:1000]

"First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you know Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us kill him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be done: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citizens, the patricians good.\nWhat authority surfeits on would relieve us: if they\nwould yield us but the superfluity, while it were\nwholesome, we might guess they relieved us humanely;\nbut they think we are too dear: the leanness that\nafflicts us, the object of our misery, is as an\ninventory to particularise their abundance; our\nsufferance is a gain to them Let us revenge this with\nour pikes, ere we become rakes: for the gods know I\nspeak this in hunger 